In [ ]:
#- `notebook.ipynb`: main code
#- `dataset.csv`: input data
#- `README.md`: project overview & diagram


In [ ]:
!pip install bokeh geopy

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset.csv


In [ ]:
import pandas as pd
df = pd.read_csv("dataset.csv")


In [ ]:
# Dynamic Pricing for Urban Parking Lots
# Capstone Project - Summer Analytics 2025

# =====================================
# 📘 SECTION 1: Imports and Data Loading
# =====================================
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Legend
output_notebook()

# Load the dataset
# Assuming the dataset is in your Google Drive
df = pd.read_csv("/content/dataset.csv")

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Encode categorical variables
df['TrafficScore'] = df['TrafficConditionNearby'].map({'low': 0.3, 'medium': 0.6, 'high': 1.0})
df['VehicleWeight'] = df['VehicleType'].map({'bike': 0.5, 'car': 1.0, 'truck': 1.5})

# Initialize base price
df['BasePrice'] = 10

In [ ]:
# 🔹 SECTION 2: Baseline Linear Model
# =====================================
def linear_model(occupancy, capacity, prev_price, alpha=0.5):
    return prev_price + alpha * (occupancy / capacity)

df['LinearPrice'] = df.groupby('SystemCodeNumber').apply(
    lambda group: group.apply(
        lambda row: linear_model(row['Occupancy'], row['Capacity'], row['BasePrice']), axis=1)).reset_index(drop=True)

/tmp/ipython-input-13-2387093475.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['LinearPrice'] = df.groupby('SystemCodeNumber').apply(


In [ ]:
# 🔹 SECTION 3: Demand-Based Model
# =====================================
def demand_function(row, alpha=0.6, beta=0.3, gamma=0.5, delta=1.0, epsilon=0.7):
    demand = (alpha * (row['Occupancy'] / row['Capacity']) +
              beta * row['QueueLength'] -
              gamma * row['TrafficScore'] +
              delta * row['IsSpecialDay'] +
              epsilon * row['VehicleWeight'])
    return demand

df['Demand'] = df.apply(demand_function, axis=1)
df['NormalizedDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())
df['DemandPrice'] = df['BasePrice'] * (1 + 0.8 * df['NormalizedDemand'])

# Bound price between 5 and 20
df['DemandPrice'] = df['DemandPrice'].clip(5, 20)


In [ ]:
# 🔹 SECTION 4: Competitive Pricing (Optional Advanced)
# =====================================
from geopy.distance import geodesic

def compute_competitive_price(row, df, radius_km=1):
    location = (row['Latitude'], row['Longitude'])
    nearby = df[(df['SystemCodeNumber'] != row['SystemCodeNumber']) &
                (df['Timestamp'] == row['Timestamp'])]
    nearby = nearby[nearby.apply(lambda x: geodesic(location, (x['Latitude'], x['Longitude'])).km < radius_km, axis=1)]
    if len(nearby) == 0:
        return row['DemandPrice']
    competitor_mean = nearby['DemandPrice'].mean()
    if row['Occupancy'] >= row['Capacity'] * 0.9 and row['DemandPrice'] > competitor_mean:
        return competitor_mean * 0.95
    elif competitor_mean > row['DemandPrice']:
        return row['DemandPrice'] * 1.1
    else:
        return row['DemandPrice']

# Apply model
sampled = df.sample(n=300)  # subset for speed
df.loc[sampled.index, 'CompetitivePrice'] = sampled.apply(
    lambda row: compute_competitive_price(row, df), axis=1)


In [ ]:
# 🔹 SECTION 5: Visualization with Bokeh
# =====================================
# Sample one parking lot
data_one_lot = df[df['SystemCodeNumber'] == df['SystemCodeNumber'].iloc[0]].copy()
data_one_lot.sort_values(by='Timestamp', inplace=True)

source = ColumnDataSource(data=dict(
    x=data_one_lot['Timestamp'],
    linear=data_one_lot['LinearPrice'],
    demand=data_one_lot['DemandPrice'],
    competitive=data_one_lot['CompetitivePrice'].fillna(0)
))

p = figure(x_axis_type='datetime', title="Dynamic Pricing Over Time", width=800, height=300)
line1 = p.line(x='x', y='linear', source=source, color='blue', legend_label='Linear Model')
line2 = p.line(x='x', y='demand', source=source, color='green', legend_label='Demand-Based Model')
line3 = p.line(x='x', y='competitive', source=source, color='red', legend_label='Competitive Model')

p.legend.location = 'top_left'
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price ($)'

show(p)

# Pathway real-time simulation/demo will go here